# This notebook contains the experiments on Adult Census dataset with LionForests

In [1]:
import pandas as pd 
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
from sklearn.preprocessing import MinMaxScaler,StandardScaler, LabelEncoder
from sklearn import preprocessing
from lionforests import LionForests

Firstly, we load the dataset and we set the feature and class names

In [2]:
feature_names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country','salary']
class_names=['<=50K','>50K'] #0: <=50K and 1: >50K
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', names=feature_names, delimiter=', ')
data_test = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test', names=feature_names, delimiter=', ')
data_test = data_test.drop(data_test.index[[0]])

/Users/johnmollas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/johnmollas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


We are doing the following preprocessing influenced by a github notebook

In [3]:
data = data[(data != '?').all(axis=1)]
data_test = data_test[(data_test != '?').all(axis=1)]
data_test['salary'] = data_test['salary'].map({'<=50K.': '<=50K', '>50K.': '>50K'})
frames = [data, data_test]
data = pd.concat(frames)

/Users/johnmollas/anaconda3/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


Feature Engineering from:
https://github.com/pooja2512/Adult-Census-Income/blob/master/Adult%20Census%20Income.ipynb. So run and skip the next code block

In [4]:
hs_grad = ['HS-grad','11th','10th','9th','12th']
elementary = ['1st-4th','5th-6th','7th-8th']
# replace elements in list.
for i in hs_grad:
    data['education'].replace(i , 'HS-grad', regex=True , inplace=True)
for e in elementary:
    data['education'].replace(e , 'elementary-school', regex=True, inplace = True)

married= ['Married-spouse-absent','Married-civ-spouse','Married-AF-spouse']
separated = ['Separated','Divorced']
#replace elements in list.
for m in married:
    data['marital-status'].replace(m ,'Married', regex=True, inplace = True)
for s in separated:
    data['marital-status'].replace(s ,'Separated', regex=True, inplace = True)

self_employed = ['Self-emp-not-inc','Self-emp-inc']
govt_employees = ['Local-gov','State-gov','Federal-gov']
for se in self_employed:
    data['workclass'].replace(se , 'Self_employed', regex=True, inplace = True)
for ge in govt_employees:
    data['workclass'].replace(ge , 'Govt_employees', regex=True, inplace = True)

del_cols = ['relationship','education-num']
data.drop(labels = del_cols, axis = 1, inplace = True)

index_age = data[data['age'] == 90].index
data.drop(labels = index_age, axis = 0, inplace =True)
num_col_new = ['age','capital-gain', 'capital-loss',
       'hours-per-week','fnlwgt']
cat_col_new = ['workclass', 'education', 'marital-status', 'occupation',
               'race', 'sex','salary','native-country']#add native-country label
scaler = MinMaxScaler()
#pd.DataFrame(scaler.fit_transform(data[num_col_new]),columns = num_col_new)
class DataFrameSelector(TransformerMixin):
    def __init__(self,attribute_names):
        self.attribute_names = attribute_names
    def fit(self,X,y = None):
        return self
    def transform(self,X):
        return X[self.attribute_names]
class num_trans(TransformerMixin):
    def __init__(self):
        pass
    def fit(self,X,y=None):
        return self
    def transform(self,X):
        df = pd.DataFrame(X)
        df.columns = num_col_new 
        return df
pipeline = Pipeline([('selector',DataFrameSelector(num_col_new)),  
                     ('scaler',MinMaxScaler()),('transform',num_trans())])#('scaler',MinMaxScaler()),        
num_df = pipeline.fit_transform(data)
num_df.shape
# columns which I don't need after creating dummy variables dataframe
cols = ['workclass_Govt_employess','education_Some-college',
        'marital-status_Never-married','occupation_Other-service',
        'race_Black','sex_Male','salary_>50K']
class dummies(TransformerMixin):
    def __init__(self,cols):
        self.cols = cols
    
    def fit(self,X,y = None):
        return self
    
    def transform(self,X):
        df = pd.get_dummies(X)
        df_new = df[df.columns.difference(cols)] 
        return df_new
pipeline_cat=Pipeline([('selector',DataFrameSelector(cat_col_new)),
                      ('dummies',dummies(cols))])
cat_df = pipeline_cat.fit_transform(data)
cat_df['id'] = pd.Series(range(cat_df.shape[0]))
num_df['id'] = pd.Series(range(num_df.shape[0]))
final_df = pd.merge(cat_df,num_df,how = 'inner', on = 'id')
print(f"Number of observations in final dataset: {final_df.shape}")

Number of observations in final dataset: (45167, 82)


We extract the train and target data from the dataframe

In [5]:
y = final_df['salary_<=50K'].values
final_df.drop(labels = ['id','salary_<=50K'],axis = 1,inplace = True)
X = final_df.values

And we need the new onehot encoded features' names

In [6]:
feature_names = list(final_df.columns.values)
categorical_features = ['workclass', 'education', 'marital-status', 'occupation', 'race', 'sex','native-country']

We declare the name of the categorical features in order LionForests to extract more compact explanations

In [7]:
my_scaler = MinMaxScaler(feature_range=(0,1))
parameters = [{
    'max_depth': [10],
    'max_features': ['sqrt'],
    'bootstrap': [False],
    'min_samples_leaf' : [1],
    'n_estimators': [100]
}]
lf = LionForests(class_names=class_names)
lf.train(X, y, my_scaler, feature_names, None, categorical_features) #Please do not ascale data before training

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   17.5s finished


Once again, we use the build-in GridSearch of LionForests to find the best classifier for this dataset

In [8]:
number_of_estimators = lf.model.n_estimators
print("Accuracy:",lf.accuracy,", Number of estimators:",lf.number_of_estimators)
print(lf.model)

Accuracy: 0.8870718533523864 , Number of estimators: 100
RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
                       max_depth=10, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)


Now we are ready to extract explanations about an instance. We choose the eleventh instance:

In [9]:
lf.following_breadcrumbs(X[10], False, True, False, complexity=4)

'if marital-status_Married & sex_Female & education_HS-grad & workclass_Private & 0.012<=fnlwgt<=0.101 & 0.404<=age<=0.486 & 0.082<=hours-per-week<=0.24 & 0.0<=capital-gain<=0.022 & 0.0<=capital-loss<=0.432 & native-country_Jamaica then >50K'

The original rule would have been this one:

In [10]:
lf.following_breadcrumbs(X[10], False, False, False, complexity=4)

'if marital-status_Married & sex_Female & education_HS-grad & workclass_Private & 0.055<=fnlwgt<=0.1 & 0.404<=age<=0.486 & 0.117<=hours-per-week<=0.24 & 0.0<=capital-gain<=0.003 & 0.0<=capital-loss<=0.366 & native-country_Jamaica then >50K'

In this example, some categorical alternative values got reduced. Currently, LionForests do not extract them automatically, but it will be implemented soon. You can notice that fnlwgt, age, hours per week and capital gain and loss values were not inverse transformed from LionForests. This is happening because we preprocessed this data externally. Thus we have to inverse transform them manually. If you want check instance 882 too.

In [11]:
pipeline.named_steps.scaler.inverse_transform([[0.486,0.003,0.336,0.24,0.25]])[0]

array([5.247800e+01, 2.999970e+02, 1.463616e+03, 2.452000e+01,
       3.827190e+05])